In [ ]:
# for similarity need to download larger english word library with the below in gitbash 
# python -m spacy download en_core_web_lg

In [ ]:
import numpy as np
import pandas as pd
import spacy
import pyspark
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()

In [ ]:
file = pd.read_csv("movie20.csv")
file_all = pd.read_csv("final_moviedata.csv")

In [ ]:
file = pd.DataFrame(file)
file.head()

In [ ]:
file_all = pd.DataFrame(file_all)
file_all.head()

In [ ]:
file_all.count()

In [ ]:
file.drop_duplicates(keep='first', inplace=True)
file_all.drop_duplicates(keep='first', inplace=True)

In [ ]:
file.count()

In [ ]:
overview_df2 = file[["imdb_id","title", "overview"]].copy()
overview_df2.dropna(inplace=True)
overview_df2.count()

In [ ]:
df_all = file_all[["imdb_id","title", "overview"]].copy()
df_all.dropna(inplace=True)
df_all.count()

In [ ]:
overview_df2.isna().any()

In [ ]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [ ]:
sentenceData = spark.createDataFrame(overview_df2.astype(str))
sentenceData.show() 

In [ ]:
sentenceData_all = spark.createDataFrame(df_all.astype(str))
sentenceData_all.show() 

In [ ]:
tokenizer = Tokenizer(inputCol="overview", outputCol="words")
tokenizer

In [ ]:
# Transform and show DataFrame
tokenized = tokenizer.transform(sentenceData)
tokenized_all = tokenizer.transform(sentenceData_all)
tokenized.show(truncate=False)

In [ ]:
# Instantiate Remover
remover = StopWordsRemover(inputCol="words", outputCol="nostopwords")

In [ ]:
# Transform and show data
tokenized = remover.transform(tokenized)
tokenized_all = remover.transform(tokenized_all)
tokenized.show(truncate=True)

In [ ]:
tokenized_all.show(truncate=True)

# Starting Spacy Similarity code

In [ ]:
# Load English tokenizer, tagger, parser, NER and word vectors
#MAKE SURE THIS IS LARGE LIBRARY

nlp = spacy.load("en_core_web_lg")



## adding columns for similarities

In [ ]:
from pyspark.sql.functions import lit

In [ ]:
#Code to compare each movie to the all movie and recommend top 3
ll = []
result = []
for m in tokenized.collect():
    main_row = str(m.nostopwords)
    docA = nlp(main_row)
    for f in tokenized_all.collect():

        overview_row = str(f.nostopwords)
        #print(overview_row)
        doc_row = nlp(overview_row)

        compare = docA.similarity(doc_row)
        ll.append({'imdb_id': f.imdb_id,'Title': f.title,
                   'Similarity': compare })
# sample = tokenized.withColumn('similarity', lit(str(ll)))
# ll   
    df_simi = pd.DataFrame(ll)
    df_simi.sort_values(by=['Similarity'], inplace=True, ascending=False)
    df_simi.reset_index(drop=True)
    top3_df = df_simi.iloc[1:4]
    result.append({'imdb_id': m.imdb_id,'Title': m.title, 'Recom1': top3_df.iloc[0,2],
                  'Recom2': top3_df.iloc[1,2], 'Recom3': top3_df.iloc[2,2]})
    print(result)


# Importing recommandation to csv

In [ ]:
df_recomm = pd.DataFrame(result)
df_recomm.count()

In [ ]:
#Use this code for the first time to create the csv file
df_recomm.to_csv('/Users/ericH/Desktop/moviesDashboard/raw/recom.csv', index = None, header=True)
df_recomm.drop_duplicates(keep='first', inplace=True)

In [ ]:
#Use this code to add to csv file
# df_recomm.to_csv('/Users/ericH/Desktop/moviesDashboard/raw/recom.csv', mode='a', index = None, header=False)


In [ ]:
file_path = '/Users/ericH/Desktop/moviesDashboard/raw/recom.csv'
with open(file_path, mode='a', newline='\n') as f:
            df_recomm.to_csv(f, index = None, header=False)